In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
import matplotlib.dates as mdates
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'notebook_connected'
from datetime import timedelta

In [2]:
# Flow temperature calculation function
def calculate_target_flow_temp(lower_temp, upper_temp, lower_flow, upper_flow, outdoor_temp):
    """Heating Curve Calculation

    Args:
        lower_temp (float): lower outdoor temperature
        upper_temp (float): upper outdoor temperature
        lower_flow (float): lower flow temperature
        upper_flow (float): upper flow temperature
        outdoor_temp (float): outdoor temperature
    """
    if outdoor_temp < lower_temp:
        return upper_flow
    elif outdoor_temp > upper_temp:
        return lower_flow
    else:
        return lower_flow + ((upper_flow - lower_flow) / (upper_temp - lower_temp)) * (upper_temp - outdoor_temp)

In [3]:
df = pd.read_csv('simulation_results_finished.csv')
df['Date'] = pd.to_datetime(df['Date'])  # Convert 'Date' column to datetime
df.set_index('Date', inplace=True)
outdoor_temp_series = df['Dry-bulb temperature (°C)']
outdoor_temp_series = outdoor_temp_series.to_frame().reset_index()

# Convert time to seconds for simulation alignment instead of half-hourly intervals
outdoor_temp_series["time_seconds"] = (outdoor_temp_series["Date"] - outdoor_temp_series["Date"].iloc[0]).dt.total_seconds()

In [6]:
# Define constants
heater_power = 2000  # Heater power in watts (W)
room_volume = 180*2.6     # Room volume in cubic meters (m³)
air_density = 1.225  # Air density in kg/m³
specific_heat_air = 1005  # Specific heat of air in J/(kg·K)

Uvalues = pd.DataFrame({'Component': ['External wall', 'Roof', 'Floor', 'Doors', 'Windows'], 'U-value': [0.08, 0.07, 0.08, 0.5, 0.8]})
buildingAreas = pd.DataFrame({'Component': ['External wall', 'Roof', 'Floor', 'Doors', 'Windows'], 'Area': [140, 90, 90, 2, 4]})
heat_loss_rate = np.sum(np.array(Uvalues['U-value']) * np.array(buildingAreas['Area']))  # Heat loss rate to environment in W/K (depends on insulation)

# initial_indoor_temp = 20  # Initial indoor temperature in °C
# thermostat_temp = 20  # Thermostat set temperature in °C
# hysteresis = 2.0  # Thermostat hysteresis in °C
simulation_time = 365 * 24 * 3600  # Simulation time for one year (in seconds)
time_step = 1  # Time step in seconds
# air_mass_flow_rate = 0.1  # Air mass flow rate in kg/s

# Derived constants
# air_mass = room_volume * air_density  # Total air mass in the room (kg)

# Simulation variables
time = np.arange(0, simulation_time, time_step)  # Time array for one year
flow_temps = np.zeros_like(time, dtype=float)  # indoor temperature array
target_flow_temps = np.zeros_like(time, dtype=float)  # Target flow temperature array
heat_pump_status = np.zeros_like(time, dtype=int)  # Heat pump status array (1=ON, 0=OFF)
return_temps = np.zeros_like(time, dtype=float)  # Heat pump temperature array
heat_loss = np.zeros_like(time, dtype=float)  # Heat loss array
heat_power = np.zeros_like(time, dtype=float)  # Heat power array
elec_power_profile = np.zeros_like(time, dtype=float)  # Electric power consumption array
COPs = np.zeros_like(time, dtype=float)  # Coefficient of Performance array

TARGET_INDOOR_TEMP = 20

# Define heating curve parameters
lower_outdoor_temp = -20  # Lower outdoor temperature for heating curve (°C)
upper_outdoor_temp = 20  # Upper outdoor temperature for heating curve (°C)
lower_flow_temp = 20     # Lower flow temperature for heating curve (°C)
upper_flow_temp = 50     # Upper flow temperature for heating curve (°C)  

# Building heat demand calculation parameters
c_water = 4186.0            # Specific heat capacity of water in J/kg.K
rho_water = 1000.0          # Density of water in kg/m^3
flow_rate = 0.2            # Flow rate in kg/s (assumed constant for simplicity)

# Internal gains
internal_gains = 0  # Internal heat gains in W

# Interpolate outdoor temperature for simulation time steps
outdoor_temp = np.interp(time, outdoor_temp_series["time_seconds"], outdoor_temp_series['Dry-bulb temperature (°C)'])

# Initialize indoor temperature
# indoor_temp[0] = initial_indoor_temp

# Simulation loop
for t in range(0, len(time)):
    
    if outdoor_temp[t] >= TARGET_INDOOR_TEMP:
        heat_pump_status[t] = 0
        heat_loss[t] = 0
        heat_power[t] = 0
        target_flow_temp = outdoor_temp[t]
        flow_temps[t] = target_flow_temp
        return_temps[t] = target_flow_temp
        elec_power_profile[t] = 0
        Practical_COP = 0
        COPs[t] = Practical_COP
    else:
        # Calculate heat losses
        heat_loss[t] = heat_loss_rate * (TARGET_INDOOR_TEMP - outdoor_temp[t])
        
        heat_pump_status[t] = 1
        # Calculate the target flow temperature based on heat demand and outdoor temperature
        target_flow_temp = calculate_target_flow_temp(lower_temp=lower_outdoor_temp, 
                                                        upper_temp=upper_outdoor_temp, 
                                                        lower_flow=lower_flow_temp, 
                                                        upper_flow=upper_flow_temp, 
                                                        outdoor_temp=outdoor_temp[t])
        target_flow_temps[t] = target_flow_temp
        
        flow_temps[t] = target_flow_temp
        
        # Calculate heat power (W)
        heat_power[t] = max(0, heat_loss[t] - internal_gains)
        
        return_temps[t] = flow_temps[t] - ((heat_loss[t] - internal_gains) / (flow_rate * c_water))
        
        # Calculate COP
        T_condensing = flow_temps[t] + 2
        T_evaporating = outdoor_temp[t] - 6 
        
        Carnot_COP = (T_condensing + 273) / ((T_condensing+273) - (T_evaporating + 273))
        Practical_COP = 0.5 * Carnot_COP
        COPs[t] = Practical_COP
        
        elec_power = heat_power[t] / Practical_COP
        elec_power_profile[t] = elec_power/1000
        

In [ ]:
# Save the results in a DataFrame
sim_results = pd.DataFrame({
    'Time': time,
    'Outdoor Temperature (°C)': outdoor_temp,
    'Electric Power (kW)': elec_power_profile,
    'Heat Pump Status': heat_pump_status,
    'Heat Gain Power (W)': heat_power,
    'Heat Gain Energy (kWh)': (heat_power * time_step) / 3.6e6,
    'Heat Pump Status': heat_pump_status,
    'Heat Loss Power (W)': heat_loss,
    'Heat Loss Energy (kWh)': (heat_loss * time_step) / 3.6e6,
})
